## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-12-04-01-13-59 +0000,nyt,Republican Anger Erupts at Johnson as Party Fr...,https://www.nytimes.com/2025/12/03/us/politics...
1,2025-12-04-01-11-02 +0000,nypost,"Hochul, MTA hails better traffic in Manhattan ...",https://nypost.com/2025/12/03/us-news/kathy-ho...
2,2025-12-04-01-10-29 +0000,nypost,Creepy flight passenger from NJ groped seatmat...,https://nypost.com/2025/12/03/us-news/creepy-f...
3,2025-12-04-01-09-28 +0000,nyt,"Fabergé Winter Egg Sells for $30 Million, an A...",https://www.nytimes.com/2025/12/03/world/europ...
4,2025-12-04-01-09-01 +0000,nyt,Afghan Refugees Face Uncertain Future As Trump...,https://www.nytimes.com/2025/12/03/us/afghan-r...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2340/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
43,trump,59
147,new,17
112,he,14
175,ukraine,12
274,will,12


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
381,2025-12-03-02-00-00 +0000,wsj,"Pete Hegseth, who won over President Trump by ...",https://www.wsj.com/politics/national-security...,127
109,2025-12-03-22-05-00 +0000,wsj,"Rep. Elise Stefanik, a top Trump ally, critici...",https://www.wsj.com/politics/elise-stefanik-mi...,112
387,2025-12-03-01-41-00 +0000,wsj,President Trump on Tuesday lashed out against ...,https://www.wsj.com/politics/policy/ice-plans-...,111
21,2025-12-04-00-05-45 +0000,nyt,Trump Faces Choices on Russia-Ukraine Talks as...,https://www.nytimes.com/2025/12/03/us/politics...,105
375,2025-12-03-02-19-03 +0000,cbc,Trump says National Guard will soon go to New ...,https://www.cbc.ca/news/world/trump-new-orlean...,103


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
381,127,2025-12-03-02-00-00 +0000,wsj,"Pete Hegseth, who won over President Trump by ...",https://www.wsj.com/politics/national-security...
27,63,2025-12-04-00-00-00 +0000,wsj,Defense Secretary Pete Hegseth shocked officia...,https://www.wsj.com/politics/national-security...
360,56,2025-12-03-04-10-00 +0000,wsj,Republican Matt Van Epps won the special elect...,https://www.wsj.com/politics/elections/republi...
271,51,2025-12-03-13-33-53 +0000,nypost,Arrogant Putin sees ‘no point in making any se...,https://nypost.com/2025/12/03/world-news/putin...
281,45,2025-12-03-13-02-24 +0000,nypost,DHS announces New Orleans as latest city to ge...,https://nypost.com/2025/12/03/us-news/new-orle...
369,41,2025-12-03-03-10-40 +0000,bbc,YouTube says it will be less safe for kids und...,https://www.bbc.com/news/articles/cp341gz24p5o...
125,41,2025-12-03-21-20-09 +0000,nypost,MyPillow CEO Mike Lindell files paperwork to c...,https://nypost.com/2025/12/03/us-news/mypillow...
223,38,2025-12-03-16-39-00 +0000,wsj,EU Launches $3.5 Billion Plan to Secure Raw-Ma...,https://www.wsj.com/world/europe/eu-launches-3...
168,37,2025-12-03-19-37-05 +0000,latimes,Former sheriff's deputy killed after police ch...,https://www.latimes.com/california/story/2025-...
114,36,2025-12-03-21-56-13 +0000,nypost,DSA calls to infiltrate NYC government through...,https://nypost.com/2025/12/03/us-news/dsa-call...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
